In [1]:
import pandas as pd
import os
import string
from nltk.stem import WordNetLemmatizer as wn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder as le

In [2]:
csv_path = '../raw_data'
newsdf = pd.read_csv(os.path.join(csv_path, 'news.csv'))

In [3]:
newsdf

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [4]:
newsdf.tail(50).to_csv(os.path.join(csv_path, 'newsdf.csv'),sep=';') # just to see some rows on excel

In [5]:
newsdf['full'] = newsdf['title'] + ' ' + newsdf['text'] # create a new column with full text

In [6]:
newsdf['label'].value_counts()

REAL    3171
FAKE    3164
Name: label, dtype: int64

In [7]:
newsdf.drop ("Unnamed: 0", axis = 1, inplace=True) # optional
#newsdf.drop ("title", axis = 1, inplace=True)
#newsdf.drop ("text", axis = 1, inplace=True)

In [8]:
# Dividing in X and y
X = newsdf['full']
y = newsdf['label']

In [9]:
def lematize (words):
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='v')
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='r')
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='a')
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='n')
    for index, word in enumerate (words):
        words[index] = wn().lemmatize(word, pos='s')
    return ' '.join(words)

In [10]:
stop_words = stopwords.words('english')

In [11]:
def preprocessing(sentence):
    sentence = sentence.strip()
    
    sentence = sentence.lower()
    
    sentence = ''.join(char for char in sentence if not char.isdigit())
    
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, ' ') 
    
    sentence = sentence.strip()
    
    sentence = word_tokenize (sentence)
    for word in sentence:
        if word in stop_words:
            sentence.remove(word)
            
    lematize(sentence)
            
    return ' '.join(sentence)

In [12]:
X = X.str.replace ("’", "'", regex = False)
X = X.str.replace ("‘", "'", regex = False)
X = X.str.replace ('“', '"', regex = False)
X = X.str.replace ('”', '"', regex = False)

In [13]:
X_proc = X.apply(preprocessing)

In [14]:
X_proc # to see final product of preprocessing

0       smell hillary fear daniel greenfield shillman ...
1       watch exact moment paul ryan commit political ...
2       kerry go paris gesture sympathy u secretary st...
3       bernie supporter twitter erupt anger dnc try w...
4       battle new york primary matter primary day new...
                              ...                        
6330    state department say can find email clinton sp...
6331    p pb stand plutocratic pentagon p pb stand plu...
6332    anti trump protester tool oligarchy informatio...
6333    ethiopia obama seek progress peace security ea...
6334    jeb bush suddenly attack trump matter jeb bush...
Name: full, Length: 6335, dtype: object

In [15]:
y_enc = pd.Series(le().fit_transform(y))

In [16]:
y_enc # to see final product of encoding

0       0
1       0
2       1
3       0
4       1
       ..
6330    1
6331    0
6332    0
6333    1
6334    1
Length: 6335, dtype: int64

In [18]:
clean_data = pd.concat((X_proc, y_enc), axis=1)
clean_data.columns = ['text', 'target']
clean_data

,text,target
0,smell hillary fear daniel greenfield shillman ...,0
1,watch exact moment paul ryan commit political ...,0
2,kerry go paris gesture sympathy u secretary st...,1
3,bernie supporter twitter erupt anger dnc try w...,0
4,battle new york primary matter primary day new...,1
...,...,...
6330,state department say can find email clinton sp...,1
6331,p pb stand plutocratic pentagon p pb stand plu...,0
6332,anti trump protester tool oligarchy informatio...,0
6333,ethiopia obama seek progress peace security ea...,1


In [19]:
clean_data.to_csv(os.path.join(csv_path, 'clean_data.csv'))